In [1]:
import pandas as pd
import folium
from pyproj import Transformer

In [2]:
stations = pd.read_csv('netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat
0,70:ee:50:17:e9:26,02:00:00:17:e2:aa,-0.472264,39.260814
1,70:ee:50:6b:35:64,02:00:00:6b:78:60,-0.472086,39.264747
2,70:ee:50:3c:dd:7e,02:00:00:3c:fb:52,-0.467302,39.260201
3,70:ee:50:a4:63:6e,02:00:00:a4:1c:50,-0.416428,39.292213
4,70:ee:50:84:95:f6,02:00:00:84:7c:38,-0.290545,39.303333


In [3]:
zgz_coords = (41.653858, -0.876258)
map = folium.Map(location=zgz_coords, zoom_start=13)

# Iterate over each station
for index, station in stations.iterrows():
    lon = station['lon']
    lat = station['lat']

    # Add a marker for the point
    folium.Marker([lat, lon], popup=station['device_id']).add_to(map)

# If running in a Jupyter Notebook, display the map inline
map

In [4]:
'''ref_stations = [
    '70:ee:50:96:b5:46', # Xalets de Lladró
    '70:ee:50:7f:f2:14', # Bungalows Park Albufera
    '70:ee:50:17:9c:76', # Urbanització El Pedregal
    '70:ee:50:74:27:1a', # Urbanització Monte Rosado
    '70:ee:50:19:22:84'  # Serra Perenxisa
]'''

"ref_stations = [\n    '70:ee:50:96:b5:46', # Xalets de Lladró\n    '70:ee:50:17:9c:76', # Urbanització El Pedregal\n    '70:ee:50:74:27:1a', # Urbanització Monte Rosado\n    '70:ee:50:7f:f2:14', # Bungalows Park Albufera\n    '70:ee:50:19:22:84'  # Serra Perenxisa\n]"

### Generate shapefile

In [5]:
import geopandas as gpd
from shapely.geometry import Point

In [6]:
# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(stations['lon'], stations['lat'])]
gdf = gpd.GeoDataFrame(stations, geometry=geometry)

# Set CRS and save
filename = 'netatmo_stations.shp'
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs('EPSG:25830')
gdf.to_file(filename)

### Locate stations

In [11]:
station_id = '70:ee:50:19:22:84'

row = stations[stations['device_id'] == station_id]

# Extract lon and lat
lon, lat = row[['lon', 'lat']].values[0]

location_map = folium.Map(location=zgz_coords, zoom_start=13)
folium.Marker([lat, lon], popup=station_id).add_to(location_map)
location_map